In [ ]:
!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
from scipy.special import softmax
import re
from sklearn.model_selection import StratifiedKFold
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc

# from nlpaug.util import Action

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
df_train = pd.read_csv("/content/drive/My Drive/AVINDEPENDENCE/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/AVINDEPENDENCE/test.csv")
df_sub = pd.read_csv("/content/drive/My Drive/AVINDEPENDENCE/sample_submission.csv")

Preprocessing data in this case did help improving the score,


In [ ]:
df_train['Clean_Text'] = df_train['ABSTRACT'].apply(lambda x:' '.join(re.sub(r"[^A-Za-z0-9]"," ",x).split()))
df_test['Clean_Text'] = df_test['ABSTRACT'].apply(lambda x: ' '.join(re.sub(r"[^A-Za-z0-9]"," ",x).split()))

In [ ]:
def aug_nlp_data(text):
    aug = nas.AbstSummAug(model_path='t5-base', num_beam=3)
    augmented_text = aug.augment(text)
    return augmented_text

In [ ]:
# new_text = []
# new_labels  = []
# for i in range(len(tmp_trn)):
#     text = tmp_trn['text'][i]
#     label = tmp_trn['labels'][i]
#     new_text.append(aug_nlp_data(text))
#     new_labels.append(label)
    
# final_data = {
#     'text':new_text,
#     'labels':new_labels
# }
# aug_df = pd.DataFrame(final_data)
# tmp_trn = pd.concat([tmp_trn,aug_df])

In [ ]:
df_train['Computer Science'].value_counts()

0    12378
1     8594
Name: Computer Science, dtype: int64

In [ ]:
df_train['Physics'].value_counts()

0    14959
1     6013
Name: Physics, dtype: int64

In [ ]:
for i in df_train.columns[3:100]:
  print(df_train[i].value_counts(normalize=True))

0    0.590216
1    0.409784
Name: Computer Science, dtype: float64
0    0.713284
1    0.286716
Name: Physics, dtype: float64
0    0.732119
1    0.267881
Name: Mathematics, dtype: float64
0    0.751764
1    0.248236
Name: Statistics, dtype: float64
0    0.97201
1    0.02799
Name: Quantitative Biology, dtype: float64
0    0.988127
1    0.011873
Name: Quantitative Finance, dtype: float64
We consider the parametric learning problem where the objective of the learner is determined by a parametric loss function Employing empirical risk minimization with possibly regularization the inferred parameter vector will be biased toward the training samples Such bias is measured by the cross validation procedure in practice where the data set is partitioned into a training set used for training and a validation set which is not used in training and is left to measure the out of sample performance A classical cross validation strategy is the leave one out cross validation LOOCV where one sample is lef

In [ ]:
#Checking the mean lengths to decide what max_seq_lenght to use
df_temp = pd.concat([df_train,df_test])
df_temp['len'] = df_temp['TITLE'].apply(lambda x: len(x.split()))
df_temp['len'].describe()

count    29961.000000
mean         9.514402
std          3.601874
min          1.000000
25%          7.000000
50%          9.000000
75%         12.000000
max         40.000000
Name: len, dtype: float64

In [ ]:
df_train.rename(columns={
    'Computer Science':'CS',
    'Quantitative Biology':'QB',
    'Quantitative Finance':'QF'
},inplace=True)

In [ ]:
#Preparing data for multilabel classification
row_list = []
for index, row in df_train.iterrows():
  my_list = [row.CS,row.Physics,row.Mathematics,row.Statistics,
             row.QB,row.QF]
  row_list.append(my_list)
print(row_list)

[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0],

In [ ]:
df_train['labels'] = row_list

In [ ]:
#Train 3 models (xlm-roberta,xlnet-large,roberta-large) with the parameters as given (i.e with preprocessed data,also preprocessed_data helped a lot)
# train_temp = pd.DataFrame()
# train_temp['text'] = df_train['Clean_Text']
# train_temp['labels'] = df_train['labels']
#Train 1 roberta-large model without preprocessed text
train_temp = pd.DataFrame()
train_temp['text'] = df_train['TITLE']
train_temp['labels'] = df_train['labels']

Train the model with the Given Specs

In [ ]:
model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6,args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 2, 'max_seq_length': 45,'manual_seed': 2,'overwrite_output_dir': True})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [ ]:
model.train_model(train_temp)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
test_temp = pd.DataFrame()
test_temp['text'] = df_test['TITLE']

In [ ]:
test_temp['labels'] = [[0,0,0,0,0,0] for i in range(len(test_temp))]

In [ ]:
test_temp

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_temp)

In [ ]:
model_outputs

array([[0.4198838 , 0.01835321, 0.17535159, 0.90554357, 0.00580761,
        0.00139444],
       [0.00415505, 0.9954978 , 0.00462463, 0.00507491, 0.00306167,
        0.00270142],
       [0.98480266, 0.00953631, 0.09939247, 0.01843814, 0.00175147,
        0.00161416],
       ...,
       [0.44249743, 0.03486926, 0.00662019, 0.46015462, 0.6116679 ,
        0.00397776],
       [0.06208298, 0.01295382, 0.23164135, 0.9634622 , 0.00486965,
        0.00560221],
       [0.09353497, 0.00562603, 0.9769199 , 0.00354314, 0.00184231,
        0.00165493]], dtype=float32)

In [ ]:
import pickle
with open('/content/drive/My Drive/AVINDEPENDENCE/xlm_roberta.pkl','rb') as f:
  model_xlm = pickle.load(f)

In [ ]:
import pickle
with open('/content/drive/My Drive/AVINDEPENDENCE/roberta_large_preds_text_pre_epoch2.pkl','rb') as f:
  model_outputs_large_pre = pickle.load(f)

In [ ]:
import pickle
with open('/content/drive/My Drive/AVINDEPENDENCE/xlnet_large_preds.pkl','rb') as f:
    model_outputs_large = pickle.load(f)

In [ ]:
import pickle
with open('/content/drive/My Drive/AVINDEPENDENCE/roberta_large_preds.pkl','rb') as f:
  model_outputs_large_no_pre = pickle.load(f)

In [ ]:
import pickle
with open('/content/drive/My Drive/AVINDEPENDENCE/roberta_large_extended.pkl','rb') as f:
  model_outputs_large_new = pickle.load(f)

In [ ]:
model_outputs_fin = (model_outputs_large * 0.6 + model_xlm*0.4)*0.5 +  (model_outputs_large_pre * 0.5 + model_outputs_large_no_pre*0.5) * 0.5

In [ ]:
def return_whole_vals(x):
  fin = []
  for i in x:
    if i > 0.40:
      fin.append(1)
    else:
      fin.append(0)
  return fin

In [ ]:
preds = [return_whole_vals(x) for x in model_outputs_fin]

In [ ]:
sub_df = pd.DataFrame(preds, columns=['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance'])

In [ ]:
sub_df['ID'] = df_sub['ID']

In [ ]:
sub_df['Quantitative Finance'].value_counts(normalize=True)

0    0.988542
1    0.011458
Name: Quantitative Finance, dtype: float64

In [ ]:
sub_df

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,ID
0,1,0,0,1,0,0,20973
1,0,1,0,0,0,0,20974
2,1,0,0,0,0,0,20975
3,0,1,0,0,0,0,20976
4,1,0,0,0,0,0,20977
...,...,...,...,...,...,...,...
8984,1,0,0,0,0,0,29957
8985,0,0,1,0,0,0,29958
8986,0,0,0,0,1,0,29959
8987,0,0,0,1,0,0,29960


In [ ]:

sub_df.to_csv('best.csv',index=False)

In [ ]:
#Roberta-base
Seq_len = 512 
lr = 3e-5
thresh = 0.45
microf1 = 0.833

#Roberta-large
Seq_len = 512
lr=3e-5
thresh=0.40
microf1=0.8471

#Roberta-base
with_Weights,83.83 at 0.45

#Roberta-large text pre processing 
thresh=0.40
f1 = 84.939